In [1]:
import numpy as np
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import os
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')

In [2]:
engine = create_engine('postgresql+psycopg2://{user}:{pw}@{service}:{port}/{db}'.format(
    user = 'postgres',
    pw = POSTGRES_PASSWORD,
    service = 'postgres',
    port = '5432',
    db = 'nba'
))

In [3]:
myquery = '''
SELECT *
FROM games
WHERE season=2022
'''

pd.read_sql_query(myquery, con=engine)

,game_id,game_date,ot,season
0,202202170BRK,2022-02-17,0,2022
1,202202170CHO,2022-02-17,2,2022
2,202202170LAC,2022-02-17,0,2022
3,202202170MIL,2022-02-17,0,2022
4,202202170NOP,2022-02-17,0,2022
...,...,...,...,...
878,202111190NOP,2021-11-19,0,2022
879,202111290LAC,2021-11-29,0,2022
880,202201130NOP,2022-01-13,0,2022
881,202201250PHI,2022-01-25,0,2022


In [4]:
#Exercise: From the player_game table, make a table with all players who scored more than 30 points in the game. KEep only th ecolumns wiht the game and player id, points, and three point

In [4]:
myquery = '''
SELECT 
    game_id, 
    player_id, 
    fg3, 
    fg3a,
    pts
FROM player_game
WHERE pts>30
ORDER BY pts DESC
LIMIT 15
'''
pd.read_sql_query(myquery, con = engine)

,game_id,player_id,fg3,fg3a,pts
0,202101030GSW,curryst01,8,16,62
1,202001200POR,lillada01,11,20,61
2,202008110DAL,lillada01,9,17,61
3,202001200POR,lillada01,11,20,61
4,202008110DAL,lillada01,9,17,61
5,201911300HOU,hardeja01,8,14,60
6,202101060PHI,bealbr01,7,10,60
7,202104300BOS,tatumja01,5,7,60
8,201911080POR,lillada01,7,16,60
9,201911080POR,lillada01,7,16,60


In [9]:
myquery = '''
SELECT g.game_date, p.player, pg.pts, pg.fg3, pg.fg3a
FROM player_game pg
INNER JOIN players p
    ON pg.player_id = p.player_id
INNER JOIN games g
    ON pg.game_id = g.game_id
WHERE pts > 30
ORDER BY pg.pts DESC
LIMIT 15
'''

pd.read_sql_query(myquery, con = engine)

,game_date,player,pts,fg3,fg3a
0,2021-01-03,Stephen Curry,62,8,16
1,2020-01-20,Damian Lillard,61,11,20
2,2020-08-11,Damian Lillard,61,9,17
3,2020-01-20,Damian Lillard,61,11,20
4,2020-08-11,Damian Lillard,61,9,17
5,2019-11-30,James Harden,60,8,14
6,2021-01-06,Bradley Beal,60,7,10
7,2021-04-30,Jayson Tatum,60,5,7
8,2019-11-08,Damian Lillard,60,7,16
9,2019-11-08,Damian Lillard,60,7,16


In [10]:
#In the team game table, who won each game? That will require you to join the team game to itself!
#Keep the game id, the team and opponent abbrev, the team score, and teh other team's score renamed to "opponent score"

In [16]:
myquery = '''
SELECT tg1.game_id, 
    tg1.team_abbrev, 
    tg1.opponent_abbrev, 
    tg1.team_score,
    tg2.team_score AS opponent_score,
    (tg1.team_score > tg2.team_score) AS win
FROM team_game tg1
INNER JOIN team_game tg2
    ON tg1.game_id = tg2.game_id
        AND tg1.team_abbrev = tg2.opponent_abbrev
'''

pd.read_sql_query(myquery, con = engine)

,game_id,team_abbrev,opponent_abbrev,team_score,opponent_score,win
0,202202170BRK,WAS,BRK,117,103,True
1,202202170BRK,BRK,WAS,103,117,False
2,202202170CHO,MIA,CHO,111,107,True
3,202202170CHO,CHO,MIA,107,111,False
4,202202170LAC,HOU,LAC,111,142,False
...,...,...,...,...,...,...
6389,202002270GSW,GSW,LAL,86,116,False
6390,202002290PHO,GSW,PHO,115,99,True
6391,202003010GSW,GSW,WAS,110,124,False
6392,202003030DEN,GSW,DEN,116,100,True
